In [39]:
import os
import tensorflow as tf
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.applications import Xception
from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image


In [2]:
train_dir = '/kaggle/input/fall-23-intro-to-vision-dataset/Dataset/classification/train'
val_dir = '/kaggle/input/fall-23-intro-to-vision-dataset/Dataset/classification/val'

## **Train Data**

In [3]:
folders = os.listdir(train_dir)
folders

['shark', 'Jelly', 'whale', 'tuna', 'fish']

In [4]:
train_data_augmented = ImageDataGenerator(
        rescale=1./255,
        rotation_range = 30,
        shear_range = 0.2,
        zoom_range = 0.2,
        horizontal_flip = True,
        width_shift_range=0.2,
        height_shift_range=0.2,
        fill_mode='nearest'
)
train_generator = train_data_augmented.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True 
)

Found 1162 images belonging to 5 classes.


## **Val Data**

In [5]:
valfolders = os.listdir(val_dir)
valfolders

['shark', 'Jelly', 'whale', 'tuna', 'fish']

In [6]:
val_data_generator = ImageDataGenerator(
                        rescale=1./255
)

val_generator = val_data_generator.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False  
)

Found 308 images belonging to 5 classes.


# Model

In [7]:
# Load pre-trained ResNet50 model without top classification layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional layers of ResNet50
for layer in base_model.layers:
    layer.trainable = False

In [8]:
# Create a new model on top of the pre-trained ResNet50
Xception_modelmodel = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(5, activation='softmax'))  

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,
    epochs=10,  
    validation_data=val_generator,
    validation_steps=val_generator.samples // 32
)

Epoch 1/10
36/36 [==============================] - 25s 485ms/step - loss: 6.4855 - accuracy: 0.3283 - val_loss: 1.3992 - val_accuracy: 0.5694
Epoch 2/10
36/36 [==============================] - 16s 438ms/step - loss: 1.3876 - accuracy: 0.4947 - val_loss: 1.1367 - val_accuracy: 0.5243
Epoch 3/10
36/36 [==============================] - 16s 435ms/step - loss: 1.4263 - accuracy: 0.5133 - val_loss: 1.7231 - val_accuracy: 0.4757
Epoch 4/10
36/36 [==============================] - 16s 437ms/step - loss: 1.2562 - accuracy: 0.5168 - val_loss: 0.8181 - val_accuracy: 0.7292
Epoch 5/10
36/36 [==============================] - 16s 432ms/step - loss: 1.2392 - accuracy: 0.5265 - val_loss: 0.7315 - val_accuracy: 0.7500
Epoch 6/10
36/36 [==============================] - 16s 437ms/step - loss: 1.0992 - accuracy: 0.5805 - val_loss: 0.9095 - val_accuracy: 0.6042
Epoch 7/10
36/36 [==============================] - 16s 440ms/step - loss: 1.0903 - accuracy: 0.5752 - val_loss: 0.8223 - val_accuracy: 0.6493

# CNN Model

In [9]:
# Define the model
CNN_Model = Sequential()

CNN_Model.add(Conv2D(16, (3, 3), strides=1, activation='relu', input_shape=(224, 224, 3)))
CNN_Model.add(MaxPooling2D())

CNN_Model.add(Conv2D(32, (3, 3), strides=1, activation='relu'))
CNN_Model.add(MaxPooling2D())

CNN_Model.add(Conv2D(16, (3, 3), strides=1, activation='relu'))
CNN_Model.add(MaxPooling2D())

CNN_Model.add(Flatten())

CNN_Model.add(Dense(256, activation='relu'))  # Adjusted units to match Flatten layer output
CNN_Model.add(Dense(5, activation='softmax'))  # Assuming 5 output classes, adjust if needed



In [12]:
# Compile the mode

CNN_Model.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy']) 
tensorboard_callback =ReduceLROnPlateau(
    monitor='val_accuracy', patience=3, verbose=1, factor=0.5, min_lr=0.00001)

In [ ]:
CNN_Model.summary()

In [13]:
history_CNN = CNN_Model.fit(train_generator, epochs=30, validation_data=val_generator, callbacks=[tensorboard_callback])

Epoch 1/30
37/37 [==============================] - 18s 423ms/step - loss: 1.5432 - accuracy: 0.5422 - val_loss: 0.8165 - val_accuracy: 0.7208 - lr: 0.0010
Epoch 2/30
37/37 [==============================] - 16s 424ms/step - loss: 0.9747 - accuracy: 0.6351 - val_loss: 2.0923 - val_accuracy: 0.1234 - lr: 0.0010
Epoch 3/30
37/37 [==============================] - 16s 431ms/step - loss: 0.9088 - accuracy: 0.6661 - val_loss: 0.6570 - val_accuracy: 0.7468 - lr: 0.0010
Epoch 4/30
37/37 [==============================] - 16s 423ms/step - loss: 0.7759 - accuracy: 0.7031 - val_loss: 0.5768 - val_accuracy: 0.7890 - lr: 0.0010
Epoch 5/30
37/37 [==============================] - 16s 430ms/step - loss: 0.7765 - accuracy: 0.7091 - val_loss: 0.5922 - val_accuracy: 0.7630 - lr: 0.0010
Epoch 6/30
37/37 [==============================] - 16s 422ms/step - loss: 0.7439 - accuracy: 0.7289 - val_loss: 0.5180 - val_accuracy: 0.8084 - lr: 0.0010
Epoch 7/30
37/37 [==============================] - 16s 420ms/st

## Xception

In [31]:
pretrained_base_model = Xception(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3),
    classes=5  # 5 classes: fish, jelly, shark, tuna, whale 
)

In [33]:



Xception_model = Sequential([
    pretrained_base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Flatten(),                       # Adding Flatten layer to transition from 3D to 1D tensor
    Dense(1024, activation='relu'),  # Additional Dense layer
    Dropout(0.5),
    Dense(256, activation='relu'),   # Another Dense layer
    Dense(5, activation='softmax')   # Output layer for 5 classes
])

# Compile the model
Xception_model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Fit the model with your data
Xception_history = Xception_model.fit(
    train_generator,
    epochs=30,
    batch_size=32,
    validation_data=val_generator

)

Epoch 1/30
37/37 [==============================] - 41s 593ms/step - loss: 0.7877 - accuracy: 0.6885 - val_loss: 0.3478 - val_accuracy: 0.9026
Epoch 2/30
37/37 [==============================] - 22s 588ms/step - loss: 0.2329 - accuracy: 0.9277 - val_loss: 0.1817 - val_accuracy: 0.9610
Epoch 3/30
37/37 [==============================] - 21s 560ms/step - loss: 0.1840 - accuracy: 0.9441 - val_loss: 0.2418 - val_accuracy: 0.9545
Epoch 4/30
37/37 [==============================] - 21s 550ms/step - loss: 0.1237 - accuracy: 0.9656 - val_loss: 0.3058 - val_accuracy: 0.9545
Epoch 5/30
37/37 [==============================] - 21s 559ms/step - loss: 0.1210 - accuracy: 0.9656 - val_loss: 0.7074 - val_accuracy: 0.9156
Epoch 6/30
37/37 [==============================] - 21s 566ms/step - loss: 0.0673 - accuracy: 0.9802 - val_loss: 0.4150 - val_accuracy: 0.9643
Epoch 7/30
37/37 [==============================] - 21s 564ms/step - loss: 0.0942 - accuracy: 0.9768 - val_loss: 0.3473 - val_accuracy: 0.9610

In [35]:
Xception_model.save('/kaggle/working/Xception_model.h5')

In [38]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image


def classify_moves(move_folder, saved_model_path, class_names):
    # Load the saved model
    loaded_model = load_model(saved_model_path)

    move_images = os.listdir(move_folder)
    move_images.sort()  # Sort for consistent ordering

    for move_image in move_images:
        image_path = os.path.join(move_folder, move_image)
        # Pre-processing
        img_size = 224  # Assuming the model was trained on images of this size
        img = image.load_img(image_path, target_size=(img_size, img_size))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array /= 255.0  # Rescale

        # Make predictions on the single image
        predictions = loaded_model.predict(img_array)

        # Map predictions to class labels
        predicted_class_index = np.argmax(predictions, axis=1)[0]
        predicted_class = class_names[predicted_class_index]

        print(f"Image: {move_image} - Predicted Move: {predicted_class}")

# ************************************test*********************************
move_folder_path = '/kaggle/input/test-fish/TestClassification'
saved_model_path = '/kaggle/working/Xception_model.h5'
class_names = ['Jelly', 'fish', 'shark', 'tuna', 'whale'] 

classify_moves(move_folder_path, saved_model_path, class_names)


1/1 [==============================] - 1s 1s/step
Image: 48.jpg - Predicted Move: fish
1/1 [==============================] - 0s 23ms/step
Image: 8.jpg - Predicted Move: fish
1/1 [==============================] - 0s 22ms/step
Image: v_105.jpg - Predicted Move: Jelly
1/1 [==============================] - 0s 22ms/step
Image: v_106.jpg - Predicted Move: shark
1/1 [==============================] - 0s 22ms/step
Image: v_130.jpg - Predicted Move: shark
1/1 [==============================] - 0s 22ms/step
Image: v_131.jpg - Predicted Move: shark
1/1 [==============================] - 0s 21ms/step
Image: v_137.jpg - Predicted Move: tuna
1/1 [==============================] - 0s 22ms/step
Image: v_141.jpg - Predicted Move: tuna
1/1 [==============================] - 0s 21ms/step
Image: v_169.jpg - Predicted Move: whale
1/1 [==============================] - 0s 21ms/step
Image: v_175.jpg - Predicted Move: whale


## VGG19

In [20]:
from tensorflow.keras.applications import VGG19


# Define the pre-trained VGG19 model
pretrained_base_model = VGG19(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3),
    classes=5  # 5 classes: fish, jelly, shark, tuna, whale 
)

# Freeze the pre-trained layers
for layer in pretrained_base_model.layers:
    layer.trainable = False

# Create a new model on top
VGG19_model = Sequential([
    pretrained_base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(5, activation='softmax')  # 5 classes: fish, jelly, shark, tuna, whale
])

# Compile the model
VGG19_model.compile(
    optimizer=RMSprop(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Fit the model with your data
history = VGG19_model.fit(
    train_generator,
    epochs=20,
    batch_size=32,
    validation_data=val_generator
)


Epoch 1/20
37/37 [==============================] - 18s 466ms/step - loss: 1.5609 - accuracy: 0.3296 - val_loss: 1.1145 - val_accuracy: 0.7240
Epoch 2/20
37/37 [==============================] - 17s 465ms/step - loss: 1.3424 - accuracy: 0.4587 - val_loss: 1.0129 - val_accuracy: 0.6981
Epoch 3/20
37/37 [==============================] - 17s 460ms/step - loss: 1.2144 - accuracy: 0.5327 - val_loss: 0.9357 - val_accuracy: 0.7630
Epoch 4/20
37/37 [==============================] - 17s 454ms/step - loss: 1.1193 - accuracy: 0.5998 - val_loss: 0.8278 - val_accuracy: 0.7662
Epoch 5/20
37/37 [==============================] - 17s 461ms/step - loss: 1.0630 - accuracy: 0.6093 - val_loss: 0.7658 - val_accuracy: 0.8019
Epoch 6/20
37/37 [==============================] - 17s 461ms/step - loss: 1.0290 - accuracy: 0.6377 - val_loss: 0.7224 - val_accuracy: 0.8182
Epoch 7/20
37/37 [==============================] - 17s 464ms/step - loss: 0.9633 - accuracy: 0.6738 - val_loss: 0.6946 - val_accuracy: 0.8117